<a href="https://colab.research.google.com/github/akinoriosamura/tensorflow2.0-sample/blob/master/tf2_classify_strctured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q sklearn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

!pip install -q tensorflow==2.0.0-beta1
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [6]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
df = pd.read_csv(URL)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [7]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train))
print(len(val))
print(len(test))

193
49
61


In [0]:
def df_to_dataset(df, shuffle=True, batch_size=32):
  df = df.copy()
  print(df.shape)
  labels = df.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  return ds

In [14]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

(193, 14)
(49, 14)
(61, 14)


In [15]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([74 67 51 46 51], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 1 0 0 1], shape=(5,), dtype=int32)


In [0]:
example_batch = next(iter(train_ds))[0]

In [17]:
example_batch.keys()

dict_keys(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'])

In [0]:
def demo(feature_col):
  feature_layer = layers.DenseFeatures(feature_col)
  print(feature_layer(example_batch).numpy())

In [19]:
age = feature_column.numeric_column("age")
demo(age)

[[74.]
 [67.]
 [51.]
 [46.]
 [51.]]


In [20]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 50, 55, 60, 65])
demo(age_buckets)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [27]:
thal = feature_column.categorical_column_with_vocabulary_list(
'thal', ['fixed', 'normal', 'reversible']
)

tha1_one_hot = feature_column.indicator_column(thal)
demo(tha1_one_hot)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [29]:
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[-0.35338822 -0.6333085   0.20694149 -0.01427206 -0.36476773 -0.05707074
  -0.16043018 -0.6747932 ]
 [-0.35338822 -0.6333085   0.20694149 -0.01427206 -0.36476773 -0.05707074
  -0.16043018 -0.6747932 ]
 [-0.35338822 -0.6333085   0.20694149 -0.01427206 -0.36476773 -0.05707074
  -0.16043018 -0.6747932 ]
 [-0.35338822 -0.6333085   0.20694149 -0.01427206 -0.36476773 -0.05707074
  -0.16043018 -0.6747932 ]
 [ 0.08282654 -0.55787545  0.16757514 -0.048522    0.17063098 -0.371546
   0.3261821   0.08087001]]
